In [1]:
import numpy as np
import pandas as pd
import mrcfile
import torch
import matplotlib.pyplot as plt
from scipy.ndimage import rotate
from scipy.spatial.transform import Rotation

import napari
import torch_fourier_rescale

## Load in volume and downscale

A pre-simulated volume is loaded into memory, and then downscale it to a desired pixel size (for better calculation efficiency)

In [2]:
with mrcfile.open("/Users/mgiammar/Downloads/lsu.mrc", mode="r") as f:
    volume = f.data.copy()
    original_pixel_size = f.voxel_size
    original_pixel_size = original_pixel_size["x"].item()  # assuming isotropic

print(f"Original pixel size: {original_pixel_size:.3f}")
print(f"Volume shape: {volume.shape}")

# Use the torch-fourier-rescale package to rescale the volume
desired_pixel_size = 8.0
volume_downsampled, new_spacing = torch_fourier_rescale.fourier_rescale_3d(
    image=torch.from_numpy(volume),
    source_spacing=original_pixel_size,
    target_spacing=desired_pixel_size,
)

new_pixel_size = new_spacing[0].item()  # Assuming isotropic spacing
print()
print(f"New pixel size: {new_pixel_size:.3f}")
print(f"Downsampled volume shape: {volume_downsampled.shape}")

Original pixel size: 0.950
Volume shape: (512, 512, 512)

New pixel size: 8.107
Downsampled volume shape: torch.Size([60, 60, 60])


### View the volume using Napari

Doing an initial visualization of the structure in Napari without any translations or rotations

In [3]:
viewer = napari.Viewer()
viewer.add_image(
    volume_downsampled.numpy(),
    name="Downsampled Volume",
    scale=[new_pixel_size] * 3,  # Assuming isotropic spacing
)
napari.run()

## Creating slab with translations and rotations

Here, the slab is created to be the same size as the original micrograph if it were down-sampled.
This means the slab can be directly overlaid on the micrograph to visualize structure position

In [ ]:
def create_empty_slab(
    image_shape: tuple[int, int],
    image_pixel_size: float,  # Angstroms
    slab_pixel_size: float,  # Angstroms
    slab_thickness: float,  # Angstroms
) -> np.ndarray:
    """Create an empty slab with the specified pixel size and thickness.

    Parameters
    ----------
    image_shape : tuple[int, int]
        Shape of the image (height, width).
    image_pixel_size : float
        Pixel size of the image, in Angstroms.
    slab_pixel_size : float
        Desired voxel size of the slab, in Angstroms (isotropic).
    slab_thickness : float
        Thickness of the slab, in Angstroms.
    """
    # Slab spans the same physical area as the image, but will have a different
    # voxel pitch.
    height = image_shape[0] * image_pixel_size
    width = image_shape[1] * image_pixel_size

    slab_height = int(height / slab_pixel_size)
    slab_width = int(width / slab_pixel_size)
    slab_depth = int(slab_thickness / slab_pixel_size)

    slab = np.zeros((slab_height, slab_width, slab_depth), dtype=np.float32)

    return slab

2025-05-30 13:27:11.160 python[85397:132264340] +[IMKClient subclass]: chose IMKClient_Modern
2025-05-30 13:27:11.160 python[85397:132264340] +[IMKInputSession subclass]: chose IMKInputSession_Modern


### Helper function to place a smaller volume into a larger volume

In [5]:
def place_into_larger_volume(
    small_volume: np.ndarray,
    large_volume: np.ndarray,
    position: tuple[int, int, int],
    rot_x: float = 0.0,
    rot_y: float = 0.0,
    rot_z: float = 0.0,
) -> np.ndarray:
    """Transform a small volume and place it into a larger volume.

    NOTE: The rotations are applied in xyz format. Other conventions will need
    to adjust accordingly.

    Parameters
    ----------
    small_volume : np.ndarray
        The small volume to be placed, shape (depth, height, width).
    large_volume : np.ndarray
        The larger volume into which the small volume will be placed, shape
        (depth, height, width).
    position : tuple[int, int, int]
        The (x, y, z) position in the larger volume where the small volume will be placed.
    rot_x : float, optional
        Rotation angle around the x-axis in degrees, default is 0.0.
    rot_y : float, optional
        Rotation angle around the y-axis in degrees, default is 0.0.
    rot_z : float, optional
    """
    # Sequentially rotate the small volume
    volume_rotated = rotate(small_volume, rot_x, axes=(1, 2), reshape=False)
    volume_rotated = rotate(volume_rotated, rot_y, axes=(0, 2), reshape=False)
    volume_rotated = rotate(volume_rotated, rot_z, axes=(0, 1), reshape=False)

    # Calculate the position in the larger volume (only integer coordinates)
    x, y, z = position
    x_end = x + volume_rotated.shape[0]
    y_end = y + volume_rotated.shape[1]
    z_end = z + volume_rotated.shape[2]

    # Check if the small volume fits into the larger volume
    if (
        x < 0
        or y < 0
        or z < 0
        or x_end > large_volume.shape[0]
        or y_end > large_volume.shape[1]
        or z_end > large_volume.shape[2]
    ):
        print(f"Position: {position}")
        print(f"Small volume shape: {small_volume.shape}")
        print(f"Larger volume shape: {large_volume.shape}")
        raise ValueError("Volume is out of bounds!")

    # Place the rotated small volume into the larger volume
    large_volume[x:x_end, y:y_end, z:z_end] += volume_rotated

    return large_volume

In [ ]:
# slab = create_empty_slab(
#     image_shape=(4096, 4096),
#     image_pixel_size=0.936,
#     slab_pixel_size=new_pixel_size,
#     slab_thickness=2400.0,
# )

# slab.shape

(472, 472, 296)

In [ ]:
# x = np.linspace(100, 400, 4)
# y = np.linspace(100, 400, 4)

# for i in range(x.size):
#     slab = place_into_larger_volume(
#         small_volume=volume_downsampled.numpy(),
#         large_volume=slab,
#         position=(int(x[i]), int(y[i]), 100),  # Place in the first slice
#         rot_x=0.0,
#         rot_y=0.0,
#         rot_z=0.0,
#     )

In [ ]:
# # Render the slab in napari
# viewer = napari.Viewer()
# viewer.add_image(
#     slab,
#     name="Slab with Downsampled Volume",
#     scale=[new_pixel_size] * 3,  # Assuming isotropic spacing
# )
# napari.run()

## Helper function to render volume from results DataFrame

In [ ]:
def construct_slab_from_results_df(
    volume: np.ndarray,
    results_df: pd.DataFrame,
    image_shape: tuple[int, int],
    image_pixel_size: float,
    slab_pixel_size: float,
    slab_thickness: float,
):
    """Takes in a results_df DataFrame and constructs a slab for the volume."""
    slab = create_empty_slab(
        image_shape=image_shape,
        image_pixel_size=image_pixel_size,
        slab_pixel_size=slab_pixel_size,
        slab_thickness=slab_thickness,
    )
    
    # Find minimum defocus values to set zero position
    min_defocus = results_df["relative_defocus"].min()

    for i, row in results_df.iterrows():
        print(f"Placing volume {i + 1} of {len(results_df)}")
        # Get full image position and convert to slab coordinates
        position = [
            row["pos_x"] * image_pixel_size,
            row["pos_y"] * image_pixel_size,
            float(row["relative_defocus"]) - min_defocus,
            # 0.0,  # Placeholder for z position, will be set later
        ]
        position = np.array(position)
        position = np.round(position / slab_pixel_size).astype(int)
        position = tuple(position.tolist())

        orientation = [row["phi"], row["theta"], row["psi"]]

        # Convert 'ZYZ' Euler angles into 'xyz' rotation angles
        rot = Rotation.from_euler("ZYZ", orientation, degrees=True)
        rot_x, rot_y, rot_z = rot.as_euler("xyz", degrees=True)
        
        # print(f"Position: {position}, Rotations: {rot_x}, {rot_y}, {rot_z}")

        slab = place_into_larger_volume(
            small_volume=volume,
            large_volume=slab,
            position=position,
            rot_x=rot_x,
            rot_y=rot_y,
            rot_z=rot_z,
        )

    return slab

In [14]:
df = pd.read_csv("/Users/mgiammar/Downloads/large_image_test_results.csv")
df

,Unnamed: 0,particle_index,mip,scaled_mip,correlation_mean,correlation_variance,total_correlations,pos_x,pos_y,pos_x_img,...,micrograph_path,template_path,mip_path,scaled_mip_path,psi_path,theta_path,phi_path,defocus_path,correlation_average_path,correlation_variance_path
0,0,0,11.530651,11.829170,0.097247,0.966543,20598240,3027,348,3283,...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...
1,1,1,11.494925,11.242980,0.052885,1.017705,20598240,633,585,889,...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...
2,2,2,11.307406,11.123627,-0.002457,1.016743,20598240,863,1007,1119,...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...
3,3,3,11.563781,11.064187,0.009555,1.044290,20598240,1052,3214,1308,...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...
4,4,4,11.373951,10.895998,0.104812,1.034246,20598240,814,1092,1070,...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,274,274,7.683329,7.835362,0.137661,0.963027,20598240,1667,3126,1923,...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...
275,275,275,8.060508,7.819768,0.062823,1.022752,20598240,2658,1747,2914,...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/tt2DTM/large_i...,/home/mgiammar/git_repositories/

In [24]:
slab = construct_slab_from_results(
    volume=volume_downsampled.numpy(),
    results=df,
    image_shape=(4096, 4096),
    image_pixel_size=original_pixel_size,
    slab_pixel_size=new_pixel_size,
    slab_thickness=3400.0,
)

Placing volume 1 of 279
Placing volume 2 of 279
Placing volume 3 of 279
Placing volume 4 of 279
Placing volume 5 of 279
Placing volume 6 of 279
Placing volume 7 of 279
Placing volume 8 of 279
Placing volume 9 of 279
Placing volume 10 of 279
Placing volume 11 of 279
Placing volume 12 of 279
Placing volume 13 of 279
Placing volume 14 of 279
Placing volume 15 of 279
Placing volume 16 of 279
Placing volume 17 of 279
Placing volume 18 of 279
Placing volume 19 of 279
Placing volume 20 of 279
Placing volume 21 of 279
Placing volume 22 of 279
Placing volume 23 of 279
Placing volume 24 of 279
Placing volume 25 of 279
Placing volume 26 of 279
Placing volume 27 of 279
Placing volume 28 of 279
Placing volume 29 of 279
Placing volume 30 of 279
Placing volume 31 of 279
Placing volume 32 of 279
Placing volume 33 of 279
Placing volume 34 of 279
Placing volume 35 of 279
Placing volume 36 of 279
Placing volume 37 of 279
Placing volume 38 of 279
Placing volume 39 of 279
Placing volume 40 of 279
Placing v

In [25]:
# Render the slab in napari
viewer = napari.Viewer()
viewer.add_image(
    slab,
    name="Placed volumes from results",
    scale=[new_pixel_size] * 3,  # Assuming isotropic spacing
)
napari.run()